In [1]:
from ipypb import irange, track

import numpy as np

import pandas as pd

from sklearn.metrics import roc_auc_score

import lightgbm as lgb

We start by assembling the raw datasets.
We will only train and predict on the small train and test sets, respectively, so let's retrieve it.
In order to reproduce this notebook, you must place `test_small.csv` and `train_small.csv` into a sub-directory named `data` in the same directory as this notebook.

In [2]:
features = [f"var_{i}" for i in range(200)]
train_small = pd.read_csv(
    "data/train_small.csv",
    header=None,
    names=["ID_code", "target", *features],
)
train_small = train_small.set_index("ID_code")

test_small = pd.read_csv("data/test_small.csv")
test_small = test_small.set_index("ID_code")

We will store the train and test `ID_code` values in order to distinguish between these two datasets later on.

In [3]:
train_ids = train_small.index
test_ids = test_small.index

We will perform the feature engineering upon the _all_ the features in the dataset, i.e. train and test features.
Let's concatenate these two feature sets, we can retrieve the correct split later on with `train_ids` and `test_ids` defined above.

In [4]:
train_features = train_small[features]
test_features = test_small[features]
all_features = pd.concat([train_features, test_features], axis=0, sort=False)

train_targets = train_small["target"]

The Kaggle user Chris Deotte plotted the feature probability densities [here](https://www.kaggle.com/cdeotte/modified-naive-bayes-santander-0-899/notebook).
"Sibmike" later observed that these probability graphs are quite similar, as mentioned in his notebook [here](https://www.kaggle.com/sibmike/are-vars-mixed-up-time-intervals).
The only difference is that a fraction of the feature densities are flipped left/right.
By experimentation, it has been shown that if you flip all the graph such that they portray the same similar shape, LightGBM models perform better.
By arbitrary choice, we will flip those densities which have their modes on the left half side such that they have their modes on the right hand side.
This can be thought of as a data normalization.
This is implemented below, where `indeces` are those features that have been identified as being "reflected" wrt. the remaining indeces, found by visual inspection.

In [5]:
def reverse(dataframe):
    dataframe = dataframe.copy()
    indeces = [
        0, 1, 2, 3, 4, 5, 6, 7, 8, 11, 15, 16, 18, 19, 22, 24, 25, 26, 27,
        41, 29, 32, 35, 37, 40, 48, 49, 47, 55, 51, 52, 53, 60, 61, 62, 103,
        65, 66, 67, 69, 70, 71, 74, 78, 79, 82, 84, 89, 90, 91, 94, 95, 96,
        97, 99, 105, 106, 110, 111, 112, 118, 119, 125, 128, 130, 133, 134,
        135, 137, 138, 140, 144, 145, 147, 151, 155, 157, 159, 161, 162, 163,
        164, 167, 168, 170, 171, 173, 175, 176, 179, 180, 181, 184, 185, 187,
        189, 190, 191, 195, 196, 199]
    column_names = [f"var_{index}" for index in indeces]
    for column_name in column_names:
        dataframe[column_name] *= -1
    return dataframe

We will now perform this feature engineering on both the training and test features, and we are doing this in-place, forgetting about the original values.

In [6]:
reversed_features = reverse(all_features)

Now we perform a pretty standard data normalization technique, subtracting the feature mean and dividing by the standard deviation.

$$
    feature \leftarrow \frac{feature - \mathrm{mean}(feature)}{\mathrm{SD}(feature)}
$$

This results in normalized features with mean `0` and standard deviation / variance equal to `1`.

In [7]:
def scale(dataframe):
    dataframe = dataframe.copy()
    for column in dataframe:
        mean = dataframe[column].mean()
        std = dataframe[column].std()
        dataframe[column] = (dataframe[column] - mean) / std
    return dataframe

Again, we apply this to both feature sets.

In [8]:
scaled_features = scale(reversed_features)

Now on to one of the most important feature engineering tricks for the Santander Transaction Prediction dataset, used by every single top-performing entry in the original leaderboard, **feature uniqueness**.
This time, we will not take the `target` value into account at all, only focusing on feature uniqueness.
For every single original (now normalized) feature, we will calculate the number of exact duplicates.
For instance for `var_0` we will calculate a new `var_0_count` columns which is set to the number of duplicates found for that specific value of `var_0`.
Every row that shares a value for `var_0` will therefore share the same value of `var_0_count`, and `var_0_count` will be equal to the number of rows that share that specific value of `var_0`.
Let's implement this in `add_count()`.

In [9]:
def add_count(dataframe):
    new_dataframe = dataframe.copy()
    
    for column in track(list(dataframe.columns)):
        count = dataframe[column].value_counts().to_dict()
        new_dataframe[str(column) + "_count"] = dataframe[column].map(count)
        
    return new_dataframe

We will now apply this function to both splits.

In [10]:
count_features = add_count(scaled_features)

 [████████████████████████████████████████████████████████████] 200/200 [00:21<00:00, 0.11s/it]

But we will not stop here, as we did with the previous feature engineering.
This time we will add a degree of fuzzyness to the criterion of being "non-unique".
This is the first of two main ideas we will borrow from the 2nd place solution to the original competition, which can be found [here](https://www.kaggle.com/c/santander-customer-transaction-prediction/discussion/88939).
The idea is to round the features to a given number of digits before finding feature duplicates, everything else being equal to `add_count()` defined above.
We will round to `2` and `3` digits and call these two new columns per original covariate for `var_X_rounded_count_2` and `var_X_rounded_count_3` for X = 0, 1, 2, ..., 199.

In [11]:
def add_round(dataframe):
    dataframe = dataframe.copy()
    columns = [f"var_{index}" for index in range(0, 200)]
    dataframe = dataframe.copy()
    for digits in [2, 3]:
        for column in track(columns):
            rounded = dataframe[column].round(decimals=digits)
            count = rounded.value_counts().to_dict()
            dataframe[str(column) + f"_rounded_count_{digits}"] = rounded.map(count)
    return dataframe

Again, we apply this function to our dataset, resulting in two new columns per original column.

In [12]:
rounded_count_features = add_round(count_features)

 [████████████████████████████████████████████████████████████] 200/200 [00:02<00:00, 0.01s/it]

 [████████████████████████████████████████████████████████████] 200/200 [00:03<00:00, 0.02s/it]

We will now retrieve the original training and test features.

In [13]:
train_features = rounded_count_features.loc[train_ids]
test_features = rounded_count_features.loc[test_ids]

Now to the second main idea of the 2nd place solution from "Onodera" which we have had good experience in adopting for our smaller dataset.
The idea is to split every row into 200 separate rows, each new row solely containing features related to the same original feature.
For instance one row will contain `var_0`, `var_0_count`, `var_0_rounded_count_2`, `var_0_rounded_count_3`, all continuous features, and one last categorical covariate `var` which will be a categorical feature equal to `0` in the given example.

We start by defining a function which filters down a given dataset to only have those columns related to `variable` (e.g. `0`) and then adds the categorical column as well.

In [14]:
def filter_dataset(dataset, variable):
    dataset = dataset.copy()
    dataset = dataset.filter(regex=f"(^var_{variable}$|^var_{variable}_)")
    dataset.columns = list(range(dataset.shape[1]))
    dataset = dataset.assign(var=variable)
    return dataset

Now a helper function which creates such a dataset.

In [15]:
def categorical_features(dataset):
    dataset = dataset.copy()
    categorical = pd.concat(
        [filter_dataset(dataset, variable) for variable in irange(0, 200)],
        axis=0,
        sort=False,
    )
    categorical["var"] = categorical["var"].astype("category")
    return categorical

And now let's apply this to our training dataset.

In [16]:
categorical_train_features = categorical_features(train_features)

 [████████████████████████████████████████████████████████████] 200/200 [00:26<00:00, 0.13s/it]

Each original observation row has now been duplicated 200 times, one for each original feature.
We will therefore need to duplicate the targets as well.

In [17]:
duplicated_targets = pd.concat([train_targets] * 200, axis=0)

We can now create a lightgbm `Dataset` object used during training.

In [18]:
lgb_train = lgb.Dataset(
    data=categorical_train_features,
    label=duplicated_targets,
    free_raw_data=False,
)

Now the LightGBM parameters. These parameters are pretty defualt values. We started by looking at the parameters used by a lot of submissions on the original kaggle competitition. We have mainly made changes such that the training does not take so much time training. The out-of-fold validation AUC metric (which comes later) does not change drastictly when we try to change these parameters. We have increased the `learning_rate` in order to decrease the time used for training. We have set `boost` to `gbdt` because the LightGBM documentation says that it can help against overfitting. 

Changes to these parameters result in approximately 0.3 percentage points plus minus.

In [19]:
lgb_params = {
    'bagging_freq': 5,
    'bagging_fraction': 0.95,
    'boost_from_average': 'false',
    'boost': 'gbdt',
    'learning_rate': 0.05,
    'metric': 'binary_logloss',
    'min_data_in_leaf': 30,
    'min_sum_hessian_in_leaf': 10.0,
    'num_leaves': 64,
    'num_threads': 24,
    'objective': 'binary',
    'verbosity': 1,
}

The blog post given [here](https://blog.amedama.jp/entry/lightgbm-cv-model) authored by "momijiame" explains how to save the best iteration model
when using the `lgb.cv` API. The source code is used in quite a lot of Kaggle notebooks, and we have taken the source code for the given callback from there.

In [20]:
class ModelExtractionCallback:
    def __init__(self):
        self._model = None

    def __call__(self, env):
        self._model = env.model

    def _assert_called_cb(self):
        if self._model is None:
            raise RuntimeError('callback has not called yet')

    @property
    def boosters_proxy(self):
        self._assert_called_cb()
        return self._model

    @property
    def raw_boosters(self):
        self._assert_called_cb()
        return self._model.boosters

    @property
    def best_iteration(self):
        self._assert_called_cb()
        return self._model.best_iteration
    
callback = ModelExtractionCallback()

We will perform a stratified 5-fold cross validation for early stopping **and** for model evaluation.

In [21]:
from sklearn.model_selection import StratifiedKFold
NUM_SPLITS = 5
five_fold = StratifiedKFold(n_splits=NUM_SPLITS, shuffle=True, random_state=42)
folds = five_fold.split(train_features, train_targets)

We must do some index manipulation in order to translate these splits into the duplicated array.
This is required such that the same observation but with different constructed covariates is not distributed across splits.

In [22]:
modified_folds = [
    [
        np.concatenate([train_idx + i * train_features.shape[0] for i in range(0, 200)]),
        np.concatenate([val_idx + i * train_features.shape[0] for i in range(0, 200)]),
    ] for train_idx, val_idx in folds
]

We now perform cross-validated, early stopping training using `lgb.cv`.

In [23]:
result = lgb.cv(
    params=lgb_params, 
    train_set=lgb_train,
    num_boost_round=100_000,
    early_stopping_rounds=20,
    verbose_eval=20,
    folds=modified_folds,
    callbacks=[callback],
)

[20]	cv_agg's binary_logloss: 0.392425 + 3.07544e-05
[40]	cv_agg's binary_logloss: 0.335698 + 5.79904e-05
[60]	cv_agg's binary_logloss: 0.325471 + 7.34631e-05
[80]	cv_agg's binary_logloss: 0.323893 + 8.24689e-05
[100]	cv_agg's binary_logloss: 0.323676 + 8.62971e-05
[120]	cv_agg's binary_logloss: 0.323649 + 8.72517e-05
[140]	cv_agg's binary_logloss: 0.323647 + 8.73421e-05


We now use all 5 models, one for each fold in order to make predictions for each of the covariate types (200).
We also make out of fold predictions in order to evaluate the model.

In [24]:
fold_models = callback.raw_boosters
best_iteration = callback.best_iteration

validation_predictions = np.ones((len(train_features), 200))
test_predictions = np.ones((len(test_features), NUM_SPLITS, 200))

for variable in irange(0, 200):
    for fold, fold_model in enumerate(fold_models):
        validation_indeces = fold_model.valid_sets[0].used_indices
        validation_indeces = validation_indeces[: len(validation_indeces) // 200]
        
        validation_features = (
            filter_dataset(train_features, variable)
            .iloc[validation_indeces]
            .values
        )
        validation_predictions[validation_indeces, variable] = fold_model.predict(
            validation_features,
            num_iteration=best_iteration,
        )
        
        test_predictions[:, fold, variable] = fold_model.predict(
            filter_dataset(test_features, variable).values,
            num_iteration=best_iteration,
        )

 [████████████████████████████████████████████████████████████] 200/200 [05:13<00:02, 1.57s/it]

In order to combine each row that was originally split into 200 seperate rows, we combine each respective probability by calculating the products of all the odds, and the combining them together again.
We will also remove covariates entirely from the model if the ROC AUC score compared with the training set is below 0.5, i.e. worse than completely guessing.

In [25]:
validation_odds = np.ones((train_features.shape[0]))
test_odds = np.ones((test_features.shape[0], NUM_SPLITS))

for variable in irange(0, 200):
    variable_auc = roc_auc_score(
        y_true=train_targets,
        y_score=validation_predictions[:, variable],
    )
    if variable_auc >= 0.5:
        preds = validation_predictions[:, variable]
        validation_odds *= preds / (1 - preds)
        
        preds = test_predictions[:, :, variable]
        test_odds *= preds / (1 - preds)

 [████████████████████████████████████████████████████████████] 200/200 [00:03<00:00, 0.02s/it]

We now calculate the ROC AUC for the out of fold predictions.

In [26]:
combined_validation_predictions = validation_predictions / (1 + validation_predictions)
validation_auc = roc_auc_score(
    y_true=train_targets,
    y_score=combined_validation_predictions.mean(axis=1),
)
print(f"Out-of-fold AUC = {validation_auc}")

Out-of-fold AUC = 0.9065809095416483


This is the best AUC which we have gotten so far, so this will be our final model for submission.
We take the test predictions and average over all the folds, and then save the submission.

In [27]:
test_predictions = test_odds / (1 + test_odds)
test_predictions = test_predictions.mean(axis=1)
test_predictions = pd.DataFrame(test_predictions, index=test_ids, columns=["target"])
test_predictions.to_csv("submission.csv")